In [ ]:
import torch
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
!pip install uniprot
!pip install biopython
import uniprot
from Bio.SeqUtils.ProtParam import ProteinAnalysis

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Created wheel for uniprot: filename=uniprot-1.3-py3-none-any.whl size=10344 sha256=ad342d57752cf05d9e45990d03548c0e9b93e8c0e7cbfa6fed61c00771cd7627
  Stored in directory: /root/.cache/pip/wheels/06/9c/6e/c8fda92238f3ca826c2c2aff0d2c25f5677a02a9941323a463
Successfully built uniprot
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 2.3 MB 4.0 MB/s 


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
seqids, fastas = uniprot.read_fasta('/content/drive/MyDrive/uniprot_sprot.fasta')

In [ ]:
seqids[0:10]

['sp|Q6GZX4',
 'sp|Q6GZX3',
 'sp|Q197F8',
 'sp|Q197F7',
 'sp|Q6GZX2',
 'sp|Q6GZX1',
 'sp|Q197F5',
 'sp|Q6GZX0',
 'sp|Q91G88',
 'sp|Q6GZW9']

In [ ]:
fastas['sp|P09430']

{'sequence': 'MSTSRKLKSHGMRRSKSRSPHKGVKRGGSKRKYRKGNLKSRKRGDDANRNYRSHL',
 'description': 'sp|P09430 STP1_HUMAN Spermatid nuclear transition protein 1 OS=Homo sapiens OX=9606 GN=TNP1 PE=1 SV=2'}

In [ ]:
analysis = ProteinAnalysis(fastas[seqids[0]]['sequence'])

In [ ]:
analysis.molecular_weight()

29735.10070000003

In [ ]:
seqids.index("sp|Q8N6V4")

67261

In [ ]:
#Protein analysis function for obtaining measures, and linear regression model.
from sklearn.linear_model import LinearRegression
import pandas

#Preprepared protein sequence embeddings:
embeddings=pandas.read_csv('/content/drive/MyDrive/human_protein_embeddings.csv', index_col=0)


#For both aromaticity and instability, we must use the premade embeddings and 
#protein analysis variables to grade a linear regression model

#Aromaticity
#

aroma_data=[]
bad_indices=[]
#Get the sequences from the fasta
for embedding_seqid in embeddings.index:
  sequence=fastas[embedding_seqid]['sequence']
  

  analysis_data=ProteinAnalysis(sequence)
  #Try catch in case of key error
  try:
    aroma_data+=[analysis_data.aromaticity()]
    
  except KeyError:
    #Get the bad indices
    bad_indices.append(embedding_seqid)
  
#Drop the bad indices
important_embeddings2=embeddings.drop(bad_indices)

#Split into training and testing
np.random.seed(8)
all_indices = list(range(len(important_embeddings2)))
train_split = np.random.choice(all_indices, int(len(important_embeddings2)*.8), replace=False)
test_split = list(set(all_indices) - set(train_split))
train_tokens = important_embeddings2.iloc[train_split,:]

#Turn aromaticity data to numpy arrays for fit
train_y = np.array(aroma_data)[train_split]
test_tokens = embeddings.iloc[test_split]
test_y = np.array(aroma_data)[test_split]



linear_regression_model=LinearRegression().fit(np.array(train_tokens), train_y)
#predict with model

#score
print(linear_regression_model.score(test_tokens,test_y))


#Instability
#

stability_data=[]
bad_indices=[]

#Get the fasta sequences that correspond to the embeddings we prepared
for embedding_seqid in embeddings.index:
  sequence=fastas[embedding_seqid]['sequence']

  instability_data=ProteinAnalysis(sequence)
  #Instability index function throws a key error on 3 of the sequences, 
  #so they must be removed
  try:
    #Get the instability values for each sequence and add them to stability data
    stability_data+=[instability_data.instability_index()]
    
  except KeyError:
    #catalogue bad indices
    bad_indices.append(embedding_seqid)
  
  
#drop the bad indices
important_embeddings=embeddings.drop(bad_indices)

#Split into training and testing
np.random.seed(8)
all_indices = list(range(len(important_embeddings)))
train_split = np.random.choice(all_indices, int(len(important_embeddings)*.8), replace=False)
test_split = list(set(all_indices) - set(train_split))
train_tokens = important_embeddings.iloc[train_split,:]

#Turn the stability data to numpy array data to fit the model
train_y = np.array(stability_data)[train_split]
test_tokens = embeddings.iloc[test_split]
test_y = np.array(stability_data)[test_split]


linear_regression_model=LinearRegression().fit(np.array(train_tokens), train_y)
#predict with model

#score
print(linear_regression_model.score(test_tokens,test_y))








/usr/local/lib/python3.7/dist-packages/sklearn/base.py:444: UserWarning: X has feature names, but LinearRegression was fitted without feature names
  f"X has feature names, but {self.__class__.__name__} was fitted without"


0.7422303759579931


/usr/local/lib/python3.7/dist-packages/sklearn/base.py:444: UserWarning: X has feature names, but LinearRegression was fitted without feature names
  f"X has feature names, but {self.__class__.__name__} was fitted without"


-0.8188177380428985


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:128: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:130: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.


ValueError: ignored

In [ ]:
linear_regression_model.predict(train_tokens)[0:10]

array([53.56825596, 40.9159887 , 49.25788119, 68.80862069, 37.44380776,
       54.05467123, 46.81571429, 41.75217391, 60.16086957, 31.67750383])

In [ ]:
train_y[0:10]

array([53.56825596, 40.9159887 , 49.25788119, 68.80862069, 37.44380776,
       54.05467123, 46.81571429, 41.75217391, 60.16086957, 31.67750383])

In [ ]:
model, alphabet = torch.hub.load("facebookresearch/esm:main", "esm1b_t33_650M_UR50S")

Downloading: "https://github.com/facebookresearch/esm/zipball/main" to /root/.cache/torch/hub/main.zip
Downloading: "https://dl.fbaipublicfiles.com/fair-esm/models/esm1b_t33_650M_UR50S.pt" to /root/.cache/torch/hub/checkpoints/esm1b_t33_650M_UR50S.pt


In [ ]:
batch_converter = alphabet.get_batch_converter()
model.eval()  # disables dropout for deterministic results

# Prepare data (first 2 sequences from ESMStructuralSplitDataset superfamily / 4)
data = [
    ("protein1", "MKTVRQERLKSIVRILERSKEPVSGAQLAEELSVSRQVIVQDIAYLRSLGYNIVATPRGYVLAGG"),
    ("protein2", "KALTARQQEVFDLIRDHISQTGMPPTRAEIAQRLGFRSPNAAEEHLKALARKGVIEIVSGASRGIRLLQEE"),
    ("protein2 with mask","KALTARQQEVFDLIRD<mask>ISQTGMPPTRAEIAQRLGFRSPNAAEEHLKALARKGVIEIVSGASRGIRLLQEE"),
    ("protein3",  "K A <mask> I S Q"),
]
batch_labels, batch_strs, batch_tokens = batch_converter(data)

# Extract per-residue representations (on CPU)
with torch.no_grad():
    results = model(batch_tokens, repr_layers=[33], return_contacts=True)
token_representations = results["representations"][33]

In [ ]:
# Generate per-sequence representations via averaging
# NOTE: token 0 is always a beginning-of-sequence token, so the first residue is token 1.
sequence_representations = []
for i, (_, seq) in enumerate(data):
    sequence_representations.append(token_representations[i, 1 : len(seq) + 1].mean(0))

# Look at the unsupervised self-attention map contact predictions
import matplotlib.pyplot as plt
for (_, seq), attention_contacts in zip(data, results["contacts"]):
    plt.matshow(attention_contacts[: len(seq), : len(seq)])
    plt.title(seq)
    plt.show()

In [ ]:
sequence_representations[3].shape

In [ ]:
token_representations[3].shape